In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("/Users/harrisonchase/Downloads/titanic.csv")

In [5]:
questions = [
    ("How old was the oldest person?", "df['Age'].max()")
]

In [15]:
eval("df['Age'].max()")

80.0

In [6]:
from langsmith import Client
client = Client()
dataset_name = "Dynamic Titantic CSV"

# Storing inputs in a dataset lets us
# run chains and LLMs over a shared set of examples.
dataset = client.create_dataset(
    dataset_name=dataset_name, description="Test QA over CSV",
)

In [7]:
for example in questions:
    client.create_example(
        inputs={"question": example[0]},
        outputs={"code": example[1]},
        dataset_id=dataset.id,
    )

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate

In [10]:
prompt = ChatPromptTemplate.from_template("Answer the question {question}")

In [11]:
chain = prompt | ChatOpenAI()

In [23]:
from langsmith import Client
from langchain.smith import RunEvalConfig, run_on_dataset
from typing import Optional
from langchain.evaluation.criteria.eval_chain import LabeledCriteriaEvalChain

class CustomCriteriaEvalChain(LabeledCriteriaEvalChain):
    
    def _get_eval_input(
        self,
        prediction: str,
        reference: Optional[str],
        input: Optional[str],
    ) -> dict:
        raw = super()._get_eval_input(prediction, reference, input)
        # This calculates the current reference answer, and uses that instead of the code
        # This could be other things, like running a SQL query, hitting an API, etc
        raw["reference"] = eval(raw["reference"])
        return raw


def get_chain():
    return chain
    

client = Client()
eval_config = RunEvalConfig(
    custom_evaluators=[
        CustomCriteriaEvalChain.from_llm(criteria="correctness", llm=ChatOpenAI())
    ],
)
chain_results = run_on_dataset(
    client,
    dataset_name="Dynamic Titantic CSV",
    llm_or_chain_factory=get_chain,
    evaluation=eval_config,
)

View the evaluation results for project '199ea00e87b64a9ea153ada938e33521-RunnableSequence' at:
https://dev.langchain.plus/projects/p/b809b59e-3fed-4dd1-8f56-26e0955e5ca6?eval=true
